# DEMO

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'scripts'))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from math import sqrt
from embedding import adaptive_multi_level_embedding


#WPSNR funzione
def wpsnr(img1, img2):
  img1 = np.float32(img1)/255.0
  img2 = np.float32(img2)/255.0
  difference = img1-img2
  same = not np.any(difference)
  if same is True:
      return 150
  w = np.genfromtxt('csf.csv', delimiter=',')
  ew = convolve2d(difference, np.rot90(w,2), mode='valid')
  decibels = 20.0*np.log10(1.0/sqrt(np.mean(np.mean(ew**2))))
  return decibels

## Embedding

In [2]:

watermark = np.load('mediavault.npy')

image_path = './images/0099.bmp'

watermarked_image = adaptive_multi_level_embedding(image_path, watermark)

original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

image_wpsnr=wpsnr(original_image, watermarked_image)
print("WPSNR: ", image_wpsnr)

plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)  
plt.imshow(original_image, cmap='gray')
plt.title('Original Image')
plt.axis('off')  

plt.subplot(1, 3, 2) 
plt.imshow(watermarked_image, cmap='gray')
plt.title('Watermarked Image')
plt.axis('off')

plt.tight_layout()
plt.show() 


##decommenta sotto e commenta sopra per wpsnr su tutte le immagini con media

"""
watermark = np.load('mediavault.npy')
image_folder = './images'
wpsnr_values = []

for i in range(1, 101):
    image_filename = f'{i:04d}.bmp'
    image_path = os.path.join(image_folder, image_filename)
    
    if os.path.exists(image_path):
        watermarked_image = adaptive_multi_level_embedding(image_path, watermark)
        
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        image_wpsnr = wpsnr(original_image, watermarked_image)
        wpsnr_values.append(image_wpsnr)
        
        print(f"WPSNR for {image_filename}: {image_wpsnr:.2f} dB")

average_wpsnr = np.mean(wpsnr_values)
print(f"Average WPSNR: {average_wpsnr:.2f} dB") 
"""





WPSNR for 0001.bmp: 88.29 dB
WPSNR for 0002.bmp: 95.22 dB
WPSNR for 0003.bmp: 73.59 dB
WPSNR for 0004.bmp: 52.63 dB
WPSNR for 0005.bmp: 121.96 dB
WPSNR for 0006.bmp: 54.12 dB
WPSNR for 0007.bmp: 48.97 dB
WPSNR for 0008.bmp: 73.04 dB
WPSNR for 0009.bmp: 66.62 dB
WPSNR for 0010.bmp: 64.74 dB
WPSNR for 0011.bmp: 80.97 dB
WPSNR for 0012.bmp: 72.14 dB
WPSNR for 0013.bmp: 74.07 dB
WPSNR for 0014.bmp: 62.24 dB
WPSNR for 0015.bmp: 71.10 dB
WPSNR for 0016.bmp: 61.52 dB
WPSNR for 0017.bmp: 108.11 dB
WPSNR for 0018.bmp: 82.51 dB
WPSNR for 0019.bmp: 150.00 dB
WPSNR for 0020.bmp: 52.64 dB
WPSNR for 0021.bmp: 82.01 dB
WPSNR for 0022.bmp: 95.98 dB
WPSNR for 0023.bmp: 67.94 dB
WPSNR for 0024.bmp: 91.97 dB
WPSNR for 0025.bmp: 50.57 dB
WPSNR for 0026.bmp: 63.68 dB
WPSNR for 0027.bmp: 85.17 dB
WPSNR for 0028.bmp: 82.22 dB
WPSNR for 0029.bmp: 140.47 dB
WPSNR for 0030.bmp: 62.38 dB
WPSNR for 0031.bmp: 69.27 dB
WPSNR for 0032.bmp: 61.30 dB
WPSNR for 0033.bmp: 56.53 dB
WPSNR for 0034.bmp: 65.24 dB
WPSNR for 

## ROC

In [3]:
"""
# Importa la funzione dal file ROC.py
from ROC import calculate_threshold

# Specifica la cartella delle immagini e il file del watermark
images_folder = './images'
mark_file = 'mark.npy'

# Calcola la soglia ottimale
tau_optimal = calculate_threshold(images_folder, mark_file)

# Stampa la soglia ottimale
print(f"La soglia ottimale calcolata è: {tau_optimal}") """


'\n# Importa la funzione dal file ROC.py\nfrom ROC import calculate_threshold\n\n# Specifica la cartella delle immagini e il file del watermark\nimages_folder = \'./images\'\nmark_file = \'mark.npy\'\n\n# Calcola la soglia ottimale\ntau_optimal = calculate_threshold(images_folder, mark_file)\n\n# Stampa la soglia ottimale\nprint(f"La soglia ottimale calcolata è: {tau_optimal}") '